In [1]:
import datetime
FEATURE_TIME = datetime.datetime(year=2023, month=5, day=12, hour=0, minute=0, second=0)

In [2]:
import findspark
# $ cd /opt
# /opt$ sudo ln -s ~/apps/spark-3.4.0-bin-hadoop3 spark
findspark.init("/opt/spark") 

In [3]:
# ! wget -O postgresql-42.5.4.jar ~/apps/ https://jdbc.postgresql.org/download/postgresql-42.5.4.jar

In [4]:
# https://github.com/GoogleCloudDataproc/spark-bigquery-connector/releases
# ! gsutil cp gs://spark-lib/bigquery/spark-bigquery-with-dependencies_2.12-0.30.0.jar ~/apps/

In [3]:
# import pyspark
# sc = pyspark.SparkContext(appName="globalContent")
from pathlib import Path
pg_jar = str(Path('~/apps/postgresql-42.5.4.jar').expanduser().resolve())
# bq_jar = str(Path('~/apps/spark-bigquery-with-dependencies_2.12-0.30.0.jar').expanduser().resolve())
# spark_jars = f"{pg_jar},{bq_jar}"
spark_jars = pg_jar

from pyspark.sql import SparkSession
spark = SparkSession.builder \
            .appName("LensFeatures") \
            .config("spark.jars", spark_jars) \
            .getOrCreate()

23/05/20 11:25:44 WARN Utils: Your hostname, VIJAYs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.2.12 instead (on interface en0)
23/05/20 11:25:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/05/20 11:25:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
import getpass
db_host = getpass.getpass(prompt='DB Host: ')

DB Host: ········


In [7]:
db_port = getpass.getpass(prompt='DB Port: ')

DB Port: ········


In [8]:
db_name = getpass.getpass(prompt='DB Name: ')

DB Name: ········


In [9]:
db_user = getpass.getpass(prompt='DB User: ')

DB User: ········


In [10]:
db_pwd = getpass.getpass()

········


In [11]:
sql_query = """
WITH MAX_DATE AS (
  SELECT max(date) as maxdate
  FROM globaltrust
  WHERE strategy_id = 5
)
SELECT 
    ROW_NUMBER() OVER(ORDER BY v DESC) AS rank, 
    g.v AS score, 
    p.handle AS profile_handle
FROM globaltrust AS g
INNER JOIN profiles AS p ON p.id = g.i
WHERE 
  strategy_id = 5 
  AND date = (SELECT maxdate FROM MAX_DATE)
-- LIMIT 1000
"""

In [12]:
gt_df =  spark.read.format("jdbc") \
            .option("url", f"jdbc:postgresql://{db_host}:{db_port}/{db_name}") \
            .option("driver", "org.postgresql.Driver")\
            .option("user", db_user) \
            .option("password", db_pwd) \
            .option("query", sql_query) \
            .load()

In [13]:
gt_df.printSchema()

root
 |-- rank: long (nullable = true)
 |-- score: double (nullable = true)
 |-- profile_handle: string (nullable = true)



In [14]:
print(f"total number of records ${gt_df.count()}")

total number of records $103326


In [15]:
# spec_profiles_df = spark.read.format("jdbc") \
#                 .option("url", f"jdbc:postgresql://{db_host}:{db_port}/{db_name}") \
#                 .option("driver", "org.postgresql.Driver")\
#                 .option("dbtable", "profiles") \
#                 .option("user", db_user) \
#                 .option("password", db_pwd) \
#                 .load()

In [16]:
# spec_profiles_df.printSchema()

In [17]:
# print(f"total number of records ${spec_profiles_df.count()}")

In [18]:
# bq_profiles_df = spark.read.format('com.google.cloud.spark.bigquery') \
#                     .option('table','lens-public-data.polygon.public_profile') \
#                     .load()

bq_profiles_df = spark.read.parquet("/tmp/lens_bigquery_csvs/public_profile_05092023.pqt")

In [19]:
print(f"total number of records ${bq_profiles_df.count()}")

total number of records $117119


In [20]:
bq_profiles_df = bq_profiles_df.select(
    "profile_id",
    "handle"
)

In [21]:
bq_profiles_df.printSchema()

root
 |-- profile_id: string (nullable = true)
 |-- handle: string (nullable = true)



In [46]:
bq_profiles_df.groupBy("handle").count().where("count > 1").show()

+------+-----+
|handle|count|
+------+-----+
+------+-----+



In [45]:
gt_df.groupBy("profile_handle").count().where("count > 1").show()

+------------------+-----+
|    profile_handle|count|
+------------------+-----+
|nishimiyamayu.lens|    2|
| ashu87482200.lens|    2|
|              null|    3|
|      futurka.lens|    2|
|  celebration.lens|    2|
|       0xmo7d.lens|    2|
|        99220.lens|    2|
|      0xn4nx0.lens|    2|
|wguoqiang1001.lens|    2|
|       treaty.lens|    2|
|       abc123.lens|    2|
+------------------+-----+



In [49]:
# There are a few duplicate entries in the data from Spec.dev
gt_df = gt_df.dropDuplicates(['profile_handle'])

In [22]:
profile_trust_df = bq_profiles_df.join(gt_df, bq_profiles_df.handle == gt_df.profile_handle,"leftouter" )

In [23]:
profile_trust_df.printSchema()

root
 |-- profile_id: string (nullable = true)
 |-- handle: string (nullable = true)
 |-- rank: long (nullable = true)
 |-- score: double (nullable = true)
 |-- profile_handle: string (nullable = true)



In [24]:
from pyspark.sql.functions import col, when, lit
profile_trust_df = profile_trust_df.drop(col('handle')) \
                        .drop(col('profile_handle'))

In [25]:
profile_trust_df.printSchema()

root
 |-- profile_id: string (nullable = true)
 |-- rank: long (nullable = true)
 |-- score: double (nullable = true)



In [41]:
profile_trust_df.groupBy("profile_id").count().where("count > 1").show()

+----------+-----+
|profile_id|count|
+----------+-----+
|  0x01a80e|    2|
|  0x01a880|    2|
|    0x6dfd|    2|
|  0x012674|    2|
|  0x019b35|    2|
|  0x01a110|    2|
|    0x8e83|    2|
|  0x016765|    2|
|  0x0194d6|    2|
|  0x01aa70|    2|
+----------+-----+



In [26]:
# posts_df = spark.read.format('com.google.cloud.spark.bigquery') \
#                     .option('table','lens-public-data.polygon.public_profile_post') \
#                     .load()
posts_df = spark.read.csv("/tmp/lens_bigquery_csvs/public_profile_post/", 
                          header=True, 
                          escape="\"",
                          multiLine=True,
                          inferSchema=True)

In [27]:
# posts_df.show(5, truncate=False)

In [38]:
print(f"total number of records ${posts_df.count()}")

total number of records $452441


In [29]:
# joining columns with same name causes issues when querying the dataframe
# renaming profile_id column
from pyspark.sql.functions import col
posts_df = posts_df.select(
    "post_id", 
    col("profile_id").alias("post_profile_id"), 
)
# posts_df = posts_df.withColumnRenamed("profile_id", "post_profile_id")

In [30]:
posts_df.printSchema()

root
 |-- post_id: string (nullable = true)
 |-- post_profile_id: string (nullable = true)



In [31]:
profile_trust_df.printSchema()

root
 |-- profile_id: string (nullable = true)
 |-- rank: long (nullable = true)
 |-- score: double (nullable = true)



In [32]:
posts_trust_df = posts_df.join(profile_trust_df, 
                               posts_df.post_profile_id == profile_trust_df.profile_id, 
                               "leftouter")

In [33]:
posts_trust_df.printSchema()

root
 |-- post_id: string (nullable = true)
 |-- post_profile_id: string (nullable = true)
 |-- profile_id: string (nullable = true)
 |-- rank: long (nullable = true)
 |-- score: double (nullable = true)



In [34]:
posts_trust_df.filter(posts_trust_df.profile_id.isNull()).count()

0

In [35]:
print(f"total number of records ${posts_trust_df.count()}")

total number of records $452443


In [39]:
posts_trust_df.groupBy("post_id").count().where("count > 1").show()

+-------------+-----+
|      post_id|count|
+-------------+-----+
|0x012674-0x03|    2|
|0x012674-0x01|    2|
+-------------+-----+



In [47]:
posts_trust_df = posts_trust_df.dropDuplicates(['post_id'])

In [48]:
print(f"total number of records ${posts_trust_df.count()}")

total number of records $452441


In [52]:
posts_trust_df.printSchema()

root
 |-- post_id: string (nullable = true)
 |-- post_profile_id: string (nullable = true)
 |-- profile_id: string (nullable = true)
 |-- rank: long (nullable = true)
 |-- score: double (nullable = true)



In [36]:
# Credentials saved to file: [/Users/vijay/.config/gcloud/application_default_credentials.json]
# These credentials will be used by any library that requests Application Default Credentials (ADC).

# ! gcloud auth application-default login

In [37]:
PROJECT_ID = "boxwood-well-386122"
REGION = "us-central1"
BUCKET_URI = "gs://vijay-lens-feature-store-temp"  
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [50]:
FEATURESTORE_ID = "lens_featurestore_dev"
from google.cloud.aiplatform import Feature, Featurestore
fs = Featurestore(
    featurestore_name=FEATURESTORE_ID
)
print(fs.gca_resource)

name: "projects/1181216607/locations/us-central1/featurestores/lens_featurestore_dev"
create_time {
  seconds: 1683658565
  nanos: 777177000
}
update_time {
  seconds: 1683730147
  nanos: 63762000
}
etag: "AMEw9yNc_eqiIY7rz7G0-fTe0eZqwZDPu9J6jSKPio6xYa5dJTE9LSaqJbIfvKxwKow="
online_serving_config {
}
state: STABLE



In [51]:
posts_entity_type = fs.get_entity_type(entity_type_id="posts")

In [53]:
POSTS_FEATURES_IDS = ['author_f6c3m8col12enh_rank', 'author_f6c3m8col12enh_score']
POSTS_SRC_FIELDS = {
    'author_f6c3m8col12enh_rank': 'rank',
    'author_f6c3m8col12enh_score': 'score',
}
POSTS_ENTITY_ID_FIELD = "post_id"

In [54]:
POSTS_DF = posts_trust_df.toPandas()

In [55]:
POSTS_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452441 entries, 0 to 452440
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   post_id          452441 non-null  object 
 1   post_profile_id  452441 non-null  object 
 2   profile_id       452441 non-null  object 
 3   rank             450167 non-null  float64
 4   score            450167 non-null  float64
dtypes: float64(2), object(3)
memory usage: 17.3+ MB


In [56]:
# ingest_from_df(
#     feature_ids: List[str],
#     feature_time: Union[str, datetime.datetime],
#     df_source: pd.DataFrame,
#     feature_source_fields: Optional[Dict[str, str]] = None,
#     entity_id_field: Optional[str] = None,
#     request_metadata: Optional[Sequence[Tuple[str, str]]] = (),
#     ingest_request_timeout: Optional[float] = None,
# )
posts_entity_type.ingest_from_df(
    feature_ids = POSTS_FEATURES_IDS,
    feature_time = FEATURE_TIME,
    df_source = POSTS_DF,
    feature_source_fields = POSTS_SRC_FIELDS,
    entity_id_field = POSTS_ENTITY_ID_FIELD,
)

Importing EntityType feature values: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_dev/entityTypes/posts
Import EntityType feature values backing LRO: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_dev/entityTypes/posts/operations/601147193528680448
EntityType feature values imported. Resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_dev/entityTypes/posts


resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_dev/entityTypes/posts

In [ ]:
# TODO 
# 1. "recommend" label
# 2. read full dataset from bigquery
# 3. checkpoint max(block_timestamp) for incremental reads from BigQuery